### Paquetes requeridos

In [ ]:
#!pip install gymnasium
#!pip install pyvirtualdisplay
#!pip install moviepy

### Imports

In [ ]:
import glob
import io
import base64
import gymnasium

from gymnasium.wrappers.record_video import RecordVideo
from IPython.display import HTML
from IPython import display as ipythondisplay

### Funciones para grabar y mostrar video

In [ ]:
def show_video():
    """
    Utility function to enable video recording of gym environment and displaying it
    To enable video, just do "env = wrap_env(env)""
    """
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    
     
def wrap_env(env, name="default"):
    """
    Wrapper del ambiente donde definimos un Monitor que guarda la visualizacion como un archivo de video.
    """
    env = RecordVideo(env, "./video/", name_prefix=name, episode_trigger=lambda x: x % 2 == 0)
    return env

### Creamos el ambiente

In [ ]:
ENV_NAME = "Pendulum-v1"
env = wrap_env(gymnasium.make(ENV_NAME, render_mode="rgb_array"))

### Creamos un agente random

En este paso van a tener que sustitir con su agente

In [ ]:
class RandomAgent:
    def __init__(self, env):
        self.env = env
    
    def select_action(self, state):
        return self.env.action_space.sample()
    
agent = RandomAgent(env)

### Hacemos una corrida y grabamos

In [ ]:
acumulated_reward = 0
observation,_ = env.reset()
while True:
    env.render()

    action = agent.select_action(observation)
    observation, reward, done, truncated, _ = env.step(action)
    acumulated_reward += reward
    if done or truncated:
      break

print(f"Acumulated reward: {acumulated_reward}")

# Cerramos la conexion con el Monitor de ambiente y mostramos el video.
env.close()
show_video()

del env

Para agregar WandB solamente tienen que guardar el video como un artefacto. Y le pasan el path al video.

En este link pueden encontrar la documentación para hacer eso. https://docs.wandb.ai/ref/python/data-types/video

De igual forma les dejo un ejemplo:

In [ ]:
wandb.log({"Video": wandb.Video(video_path, fps=4, format="mp4")})